In [1]:
%%writefile mapper1.py

# Your code for mapper here.
import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper1.py


In [3]:
%%writefile reducer1.py

# Your code for reducer here.
import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Writing reducer1.py


In [4]:
%%writefile mapper2.py

import sys
reload(sys)

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
        print '%d\t%s' % (count, word)
    except ValueError as e:
        continue

Writing mapper2.py


In [6]:
%%writefile reducer2.py

import sys
reload(sys)

for line in sys.stdin:
    try:
        count, word = line.strip().split('\t', 1)
        count = int(count)
        print '%s\t%d' % (word, count)
    except ValueError as e:
        continue

Overwriting reducer2.py


In [17]:
! hdfs dfs -ls /data/wiki

'hdfs' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
%%bash

OUT_DIR="assignment1_wordcount_"$(date +"%s%6N")

NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

# Code for your first job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -combiner "python reducer1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

OUT_DIR1="assignment1_sorting_"$(date +"%s%6N")

NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR1} > /dev/null

# Code for your second job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming sorting" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-nr" \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python reducer2.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR1} > /dev/null

# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR1}/part-00000 | sed -n 7p
hdfs dfs -rm -r -skipTrash ${OUT_DIR1}* > /dev/null

Couldn't find program: 'bash'


In [10]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python 

In [16]:
%%cmd
dir

Microsoft Windows [Version 10.0.18363.1016]
(c) 2019 Microsoft Corporation. All rights reserved.

C:\Users\hp\coursera\06 Big Data Essentials HDFS, MapReduce and Spark RDD\Week 2>dir
 Volume in drive C has no label.
 Volume Serial Number is EA72-7350

 Directory of C:\Users\hp\coursera\06 Big Data Essentials HDFS, MapReduce and Spark RDD\Week 2

08/24/2020  02:09 PM    <DIR>          .
08/24/2020  02:09 PM    <DIR>          ..
08/24/2020  01:55 PM    <DIR>          .ipynb_checkpoints
08/24/2020  02:09 PM            11,616 01 word count.ipynb
08/24/2020  01:56 PM               490 mapper1.py
08/24/2020  01:58 PM               229 mapper2.py
08/24/2020  01:56 PM                50 reducer.py
08/24/2020  01:58 PM               486 reducer1.py
08/24/2020  01:59 PM               229 reducer2.py
               6 File(s)         13,100 bytes
               3 Dir(s)  292,133,691,392 bytes free

C:\Users\hp\coursera\06 Big Data Essentials HDFS, MapReduce and Spark RDD\Week 2>